## <ins>_Vehicle Classifier using Transfer Learning_</ins>
 This notebook demonstrates how to apply transfer learning for the task of identifying different vehicle types. The model utilized is EfficientNet B3, pre-trained on the ImageNet dataset, which consists of millions of images. This extensive training allows the model to effectively extract image features, to which an additional layer is added to adapt it for our specific use case: classifying vehicles into seven distinct categories.
***

### 1. Importing the required modules.

run `!pip install tensorflow matplotlib`

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot as plt

In [2]:
# For avoiding OOM errors (Out Of Memory).
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

### 2. Preparing Dataset.

In [ ]:
# Define the path to your dataset
dataset_dir = 'train'

# Create training and validation datasets
train_dataset = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(300, 300),
    batch_size=32
)

val_dataset = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(300, 300),
    batch_size=32
)


### 3. Model Building.

In [4]:
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# Freeze the base model layers to retain pre-trained features
base_model.trainable = False

# Add new layers for our specific classification task
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Pooling layer to reduce dimensions
x = Dense(128, activation='relu')(x)  # Fully connected layer
predictions = Dense(7, activation='softmax')(x)  # Output layer for 7 classes

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])


### 4. Training the model.

In [ ]:
# Store the models performance over the epochs for visualization and valuation.
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10  # Adjust based on your dataset
)


In [ ]:
# Fine-tuning the EfficientNet layers
base_model.trainable = True
model.compile(optimizer=Adam(learning_rate=0.00001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model again
history_finetune = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5  # Adjust based on performance
)


### 5. Evaluating the model.

In [ ]:
# Validation loss and accuracy
val_loss, val_acc = model.evaluate(val_dataset)
print(f"Validation Accuracy: {val_acc}")

In [ ]:
# Plotting the graph without fine-tuning.

# Accuracy
fig = plt.figure()
plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# Loss
fig = plt.figure()
plt.plot(history.history['accuracy'], color='teal', label='accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
# Plotting the graph after fine-tuning.

# Accuracy
fig = plt.figure()
plt.plot(history_finetune.history['loss'], color='teal', label='loss')
plt.plot(history_finetune.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# Loss
fig = plt.figure()
plt.plot(history_finetune.history['accuracy'], color='teal', label='accuracy')
plt.plot(history_finetune.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

### 6. Saving the model.

In [10]:
# Saving the model and it's parameters in keras form.
model.save('vehicle_recognition_model.keras')